In [11]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

repo_id = "letgoofthepizza/Llama-3-8B-Instruct-news-summary"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, 
    max_length=1024, 
    temperature=0.1,
)

question = "2002년 FIFA 월드컵에서 누가 우승했어?"

template = """Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

llm_chain = LLMChain(
    prompt=prompt, 
    llm=llm
)
print(llm_chain.invoke (question))

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/noah/.cache/huggingface/token
Login successful
[[Generation(text=' LLM stands for "Low-Level Master". It\'s a term used in the context of computer networks, specifically in the context of network protocols and architecture. In this context, LLM refers to a master device that controls and manages a network of devices at a low level, typically at the physical or data link layer. This is in contrast to a high-level master, which might manage the network at a higher level, such as the network layer or above. Does that make sense? 😊\n\n(If you\'d like more information or clarification, feel free to ask!) 👀\n\nEdit: I realized that LLM can also stand for "Large Language Model", which is a type of artificial intelligence model that is trained on large amounts of text data to generate human-li

In [2]:
from langchain_huggingface import HuggingFacePipeline
import torch

# CUDA 디바이스 설정
device = 0 if torch.cuda.is_available() else -1

llm = HuggingFacePipeline.from_model_id(
    model_id="letgoofthepizza/Llama-3-8B-Instruct-news-summary",
    task="text-generation",
    device=device,  # GPU 사용을 지정
    pipeline_kwargs={
        "max_new_tokens": 500,
        "top_k": 50,
        "temperature": 0.1,
    },
)

text = """최근 미국 소비자물가지수(CPI)가 둔화 조짐을 보이며 뉴욕 증시의 투자 심리를 살렸다. 물가가 잡히면 금리 인하 명분이 살아나기 때문이다. 그러나 국내 증시는 글로벌 대비 언더퍼폼(underperform·수익률 저조)하며 다소 답답한 흐름을 이어가고 있다. 전체적으로 오르지 않고 개별 업종·종목의 순환매 장세가 이어지는 분위기다. 환기가 필요한 시점이다. 조준기 SK증권 연구원은 “현재 국내 증시는 호재가 나타나면 그 부분을 어느 정도 반영하기는 하나 강도가 그리 강하진 않다”며 “코스피 지수가 한 단계 더 도약하려면 추가적인 호재가 등장해야 한다”고 말했다.

향후 상당 기간 증시 향방을 결정할 이벤트로는 어떤 게 있을까. 전문가들은 한국 시각으로 이틀 후인 23일(목요일) 새벽에 발표될 미국 반도체 기업 엔비디아의 실적을 꼽는다. 임정은 KB증권 연구원은 “인공지능(AI) 반도체 붐의 핵심인 엔비디아는 최근 증시 상승세의 강력한 동력 역할을 해왔고, 실적 기대치 또한 높아진 상황”이라고 했다.

증권가는 엔비디아의 지난 회계연도(2023년 5월~2024년 4월) 매출을 246억달러(약 34조원)으로 보고 있다. 1년 전보다 242% 늘어난 수치다. 순익 컨센서스(전망치 평균)도 128억3000만달러(약 17조원)로 전년 대비 6배가량 높다. 엔비디아는 최근 5개 분기 실적 발표에서 매출과 주당순이익(EPS) 모두 서프라이즈를 기록하며 기대에 부응했
물론 최근 2개 분기 실적 발표에서는 실제 실적과 가이던스 모두 예상치를 웃돌았는데도 차익 실현 매물이 출회한 바 있다. 이번에도 호실적을 발표했는데, 주가는 하락하는 것이 아닐까. 실제 미국에서는 엔비디아가 예상치를 웃도는 실적을 발표하겠지만, 주가는 하락할 수도 있다면서 차라리 엔비디아로 인해 호황을 맞고 있는 에너지주에 투자하는 게 낫다는 투자 의견이 나오기도 했다.

그럼에도 전문가들은 일단 실적이 눈높이를 충족하는 것이 중요하다고 강조한다. 조준기 연구원은 “일시 조정 이후 다시 달렸던 경험이 있기에 일단 좋은 실적과 가이던스는 증시 추세 강화의 필수 요건이라고 생각한다”고 했다.
## 다음 뉴스 기사를 한글로 요약해줘."""

result = llm.invoke(text)
print(result)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 23.69 GiB of which 191.94 MiB is free. Including non-PyTorch memory, this process has 23.49 GiB memory in use. Of the allocated memory 23.24 GiB is allocated by PyTorch, and 1.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 

In [17]:
llm.invoke("안녕, 나는")

'안녕, 나는 30대 중반의 직장인입니다. 최근에 직장에서 일하는 것이 점점 힘들어지고 있습니다. 직장에서 일하는 것이 힘들어지면서 직장에서 일하는 것이 점점 힘들어지고 있습니다. 직장에서 일하는 것이 힘들어지면서 직장에서 일하는 것이 점점 힘들어지고 있습니다. 직장에서 일하는 것이 힘들어지면서 직장에서 일하는 것이 점점 힘들어지고'